In [ ]:
%%writefile requirements.txt
joblib
numpy
scikit-learn>=0.21.0
seldon-core
tornado>=6.0.3

In [ ]:
!pip install -r requirements.txt

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import argparse
import joblib
import logging
import sys

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

In [ ]:
class SVMClassifier(object):
    def __init__(self):
        self.model_file = "trained_svm.dat"
    
    def preprocess(self):
        data = load_iris()
        X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'])
        self.train_data = (X_train, y_train)
        self.test_data = (X_test, y_test)
        
    def train(self):
        logging.info("Training SVM")
        self.preprocess()
        model = SVC(gamma='auto')
        model.fit(self.train_data[0], self.train_data[1])
        logging.info("Model accuracy: %.2f", model.score(self.test_data[0], self.test_data[1]))
        joblib.dump(model, self.model_file)
        logging.info("Model saved as: %s", self.model_file)
        
    def predict(self, X):
        model = joblib.load(self.model_file)
        return model.predict(X)

In [ ]:
import random, string
HASH = ''.join([random.choice(string.ascii_lowercase) for n in range(16)] + [random.choice(string.digits) for n in range(16)])
AWS_REGION = 'eu-west-1'
!aws s3 mb s3://{HASH}'-kubeflow-pipeline-data' --region $AWS_REGION

In [ ]:
!pip install kubeflow-fairing

In [ ]:
!eval $(aws ecr get-login --no-include-email --region=$AWS_REGION)

In [ ]:
from kubeflow import fairing

AWS_ACCOUNT_ID = fairing.cloud.aws.guess_account_id()
DOCKER_REGISTRY = '{}.dkr.ecr.{}.amazonaws.com'.format(AWS_ACCOUNT_ID, AWS_REGION)
S3_BUCKET = f'{HASH}-kubeflow-pipeline-data'

In [ ]:
from kubeflow.fairing.builders.cluster.s3_context import S3ContextSource
from kubeflow.fairing.backends import KubeflowAWSBackend

import importlib

BuildContext = S3ContextSource(
    aws_account=AWS_ACCOUNT_ID, region=AWS_REGION,
    bucket_name=S3_BUCKET
)

backend = KubeflowAWSBackend(build_context_source=BuildContext)

In [ ]:
from kubeflow.fairing import TrainJob

train_job = TrainJob(SVMClassifier, input_files=["requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=backend)
train_job.submit()

In [ ]:
!aws s3 ls $S3_BUCKET --recursive

In [ ]:
!aws s3 rb s3://$S3_BUCKET --force
!aws ecr delete-repository --repository-name fairing-job --region $AWS_REGION --force